<a href="https://colab.research.google.com/github/CFeenan/SolarCNN/blob/master/Rotating_Months_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np


months = ["june", "july", "august", "september", "october", "november"]

month_to_data = {
    month: {
        "X": np.load(f"X_{month}.npy"),
        "y": np.load(f"y_{month}.npy")
    }
    for month in months
}

folds = [
    (["july", "august", "september", "october", "november"], "june"),
    (["june", "august", "september", "october", "november"], "july"),
    (["june", "july", "september", "october", "november"], "august"),
    (["june", "july", "august", "october", "november"], "september"),
    (["june", "july", "august", "september", "november"], "october"),
    (["june", "july", "august", "september", "october"], "november")
]


In [6]:
import torch
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self):
        super(CNN1D, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv1d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(32)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(64)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm1d(128)

        self.pool = nn.MaxPool1d(kernel_size=2)

        # Fully connected layers
        self.fc1 = nn.Linear(128 * 7, 64)  # 61 → 30 → 15 → 7 (after 3 poolings)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 1)  # Output is a single logit

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))  # Now shape: (batch, 128, 7)
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x  # No sigmoid here — handled in BCEWithLogitsLoss


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Device config
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Collect results
results = []
# Run all folds
for fold_index, (train_months, test_month) in enumerate(folds):

    print(f"\n Fold {fold_index + 1} — Test Month: {test_month}")

    # Prepare data
    X_train = np.concatenate([month_to_data[m]["X"] for m in train_months])
    y_train = np.concatenate([month_to_data[m]["y"] for m in train_months])
    X_test = month_to_data[test_month]["X"]
    y_test = month_to_data[test_month]["y"]

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64)

    model = CNN1D().to(device)
    pos_weight = torch.tensor([2.0], dtype=torch.float32).to(device)  # Fixed pos_weight for consistency
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.Adam(model.parameters(), lr=0.0005)

    # Train
    model.train()
    for epoch in range(30):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Evaluate
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs).squeeze()
            preds = (torch.sigmoid(outputs) > 0.45).int().cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    acc = accuracy_score(all_labels, all_preds)
    prec = precision_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    print(f"Accuracy: {acc:.3f} | Precision (Faulty): {prec:.3f} | Recall (Faulty): {rec:.3f} | F1 (Faulty): {f1:.3f}")



🔁 Fold 1 — Test Month: june
Accuracy: 0.891 | Precision (Faulty): 0.688 | Recall (Faulty): 0.994 | F1 (Faulty): 0.814

🔁 Fold 2 — Test Month: july
Accuracy: 0.900 | Precision (Faulty): 0.746 | Recall (Faulty): 0.883 | F1 (Faulty): 0.809

🔁 Fold 3 — Test Month: august
Accuracy: 0.947 | Precision (Faulty): 0.912 | Recall (Faulty): 0.861 | F1 (Faulty): 0.886

🔁 Fold 4 — Test Month: september
Accuracy: 0.943 | Precision (Faulty): 0.855 | Recall (Faulty): 0.917 | F1 (Faulty): 0.885

🔁 Fold 5 — Test Month: october
Accuracy: 0.952 | Precision (Faulty): 0.939 | Recall (Faulty): 0.856 | F1 (Faulty): 0.895

🔁 Fold 6 — Test Month: november
Accuracy: 0.924 | Precision (Faulty): 0.829 | Recall (Faulty): 0.861 | F1 (Faulty): 0.845
